In [ ]:
import pandas as pd
news = pd.read_csv('news_data.csv')
news['date'] = pd.to_datetime(news['date'])

group = news.groupby(['date'], as_index=False)

# 取出每日熱度前十的新聞標題
group.apply(lambda df: df.sort_values(by=['hot'], ascending = False).head(10)).to_csv('sort.csv',encoding="utf-8-sig")

In [ ]:
sort = pd.read_csv('sort.csv')
dictkeys = sort['date'].unique().tolist()

In [ ]:
# 使用結巴條件將新聞標題進行斷詞
import jieba

# load stopwords set
stopword_set = set()
with open('stopwords.txt','r', encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))



output = open('news_seg.txt', 'w', encoding='utf-8-sig')
for sentence in sort['title'].values:
    words = jieba.cut(sentence)
    for word in words:
        if word not in stopword_set:
            output.write(word + ' ')
    output.write('\n')
    #print('/'.join(seg_list))

In [ ]:
# 使用Stanza套件取出重要詞性(名詞、動詞、形容詞)
import stanza

nlp = stanza.Pipeline(lang="zh")
news_seg_file = open('news_seg.txt','r', encoding='utf-8-sig')
word2vec_training_file = open('word2vec_training.txt', 'w', encoding='utf-8-sig')

for sentence in news_seg_file:

    # 詞性標記
    doc = nlp(sentence)

    for sentence in doc.sentences:
        for word in sentence.words:
            if word.pos == "NOUN" or word.pos == "VERB" or word.pos == "ADJ":
                word2vec_training_file.write(word.text + ' ')
            
    word2vec_training_file.write('\n')      
    
word2vec_training_file.close()   

In [ ]:
# 產生word2vec詞向量轉換結果
from gensim.models import word2vec
sentences = word2vec.LineSentence("word2vec_training.txt")
model = word2vec.Word2Vec(sentences, vector_size=100)
model.save("word2vec.model")
model = word2vec.Word2Vec.load("word2vec.model")

In [ ]:
txt = pd.read_csv('word2vec_training_2.txt', delimiter = "\t", header = None)
txt

In [ ]:
finaldf = pd.concat([sort, txt], axis=1)
finaldf['date'] = pd.to_datetime(finaldf['date'])
finaldf = finaldf.groupby('date').sum(0)

In [ ]:
word_split = finaldf[0].apply(lambda x: x.split())
word_split

In [ ]:
final_list = []
for words in word_split:
    temp_list = []
    for word in words:
        if word in model.wv.key_to_index:
            temp_list.append(model.wv[word].tolist())
    final_list.append(temp_list)


In [ ]:
finalword2vec = pd.DataFrame(final_list,index = dictkeys).fillna(0)
finalword2vec.to_csv('word2vec.csv')